# Newton-Fraktal II

In den vorherigen beiden Notebooks haben Sie Code erarbeitet, um das Newton-Verfahren anwenden zu können und die Newton-Fraktale zu plotten.
In diesem Notebook wollen wir, mit den bisherigen Ergebnissen, tiefergehende Fragen über Newton-Fraktale beantworten.

## Methoden aus dem vorherigen Notebook

Dafür wollen wir zunächst den Code aus den ersten beiden Notebooks sammeln.

Dafür importieren wir zunächst die benötigten Bibliotheken.

In [ ]:
# ein paar nützliche imports
import matplotlib.pyplot as plt
import numpy as np
import random

Nun der Code für das Newton-Verfahren und um die Nullstellen zu ermitteln. Die Funktion <code>newton_fract</code> beschreibt das Newton-Verfahren. Dabei liefert diese auch die Anzahl der Iterationen zurück und markiert in <code>z</code> die nicht-konvergenten Stellen mit <code>np.inf</code>.

Um die Nullstellen der Funtion zu finden, benötigen wir die Funktion <code>find_roots</code>.

In [ ]:
def newton_fract(fun,derivative,real_lim=(-1,1),imag_lim=(-1,1),max_iter=40,eps=0.001):
    """
    fun: die Funktion
    derivative: die Ableitung der Funktion
    real_lim,imag_lim: der Auswertungsbereich
    max_iter: Anzahl der Iterationen
    eps: ist |z_i-z_{i+1}|<eps, so sehen wir z_i als konvergiert an

    Rückgabe
    rr,ii: die Realteile und Imaginärteile des Auswertungsbereichs
    z: die Resultate der Newton-Iteration
    iterations: die Zahl der Schritte bis zur Konvergenz
    """
    resolution=500 # die Auflösung des Bildes
    reals=np.linspace(real_lim[0],real_lim[1],resolution)  # die Realteile nehmen wir aus [real_min,real_max]
    imags=np.linspace(imag_lim[0],imag_lim[1],resolution) # die Imaginärteile aus [imag_min,imag_max]
    rr,ii=np.meshgrid(reals,imags)
    z=rr+ii*1j # dies sind unsere Startwerte
    iterations=np.zeros(shape=z.shape) # Zahl der Iterationen ist zu Beginn 0, mit gleicher Form wie z

    for i in range(max_iter):
        new_z=z-fun(z)/derivative(z)
        iterations+=np.where(np.abs(z-new_z)>=eps,1,0)
        z=new_z
    z=np.where(iterations==max_iter,np.inf,z)  # mark non-converged with inf
    return rr,ii,z,iterations

def find_roots(zz,eps=0.01,K=1000):
    roots=[]
    sample=np.random.choice(zz.flat,size=1000) # zufällige Wahl, damit wir nicht alle überprüfen müssen
    for z in sample:
        if np.abs(z)<K:
            check_for_new_root(z,roots,eps=eps)
    return roots

def check_for_new_root(z,roots,eps=0.01):
    for r in roots:
        if np.abs(z-r)<eps:
            return
    roots.append(z)


Schließlich der Visualisierungscode. Die Funktion <code>show_fractal</code> liefert uns den gewünschten Plot (wie wir ihn bereits aus dem 2. Notebook kennen).






In [ ]:
import matplotlib.colors as colors

def make_lighter(colour,how_much,num_shades=10):
    if isinstance(colour,str):
        rgb=colors.to_rgb(colour)
    else:
        rgb=colour
    factor=1+how_much/num_shades*2
    return tuple(np.clip(np.array(rgb)*factor,a_min=0,a_max=1))

tab_colours=["tab:blue","tab:orange","tab:green","tab:red","tab:purple","tab:brown","tab:pink","tab:gray","tab:olive","tab:cyan"]

def get_colourmap(num_shades=40,base_colours=tab_colours):
    colourmap=[make_lighter(base_colour,i,num_shades=num_shades) for base_colour in base_colours for i in range(num_shades)]
    colourmap.append((0,0,0)) # schwarz ganz ans Ende (für später)
    return colourmap

def bin(iterations,num_bins,max_iter):
    # diese Lösung ist ein wenig komplizierter als nötig
    # np.clip ersetzt alle Werte kleiner als a_min mit a_min und alle Werte größer als a_max mit a_max
    return np.clip(iterations*num_bins//max_iter,a_min=0,a_max=num_bins-1)

def show_fractal(rr,ii,z,iterations,roots,max_iter,eps=0.01):
    num_shades=40
    colourmap=get_colourmap()
    rootmap=np.zeros(shape=z.shape)
    for i,root in enumerate(roots):
        rootmap+=np.where(np.absolute(z-root)<eps,i,0)

    values=rootmap*num_shades+bin(iterations,num_shades,max_iter)
    values=np.where(np.isinf(z),len(colourmap)-1,values) # markiere die nicht-konvergenten Punkte

    levels=np.arange(len(colourmap)+1)-0.5
    fig,ax=plt.subplots(figsize=(4,4))
    ax.contourf(rr,ii,values,levels=levels,colors=colourmap)

Zur Illustration machen wir einen hübschen Plot von $z^4+z^3+z^2+z+1$.
So können wir außerdem sehen, dass der Code das macht, was wir wollen.

Damit hier auch noch etwas Neues kommt, führen wir noch eine Kurzschreibweise für Einzeiler-Funktionen ein, *lambda-Funktionen*. Das Polynom $z^4+z^3+z^2+z+1$ können wir wie bisher so definieren:

In [ ]:
def fun1(z):
    return z**4+z**3+z**2+z+1

Kürzer geht's als *lambda-Funktion*:

In [ ]:
fun1=lambda z: z**4+z**3+z**2+z+1

Wir sehen: direkt nach dem Schlüsselwort <code>lambda</code> kommt die Variable, dann ein <code>:</code> und schließlich der Rückgabewert. (Sie können die Polynome auch weiterhin mit <code>def</code> definieren.)
So, nun aber endlich der Plot von $z^4+z^3+z^2+z+1$.

In [ ]:
deriv1=lambda z : 4*z**3+3*z**2+2*z+1

rr,ii,z,iterations=newton_fract(fun1,deriv1,real_lim=(-1,1),imag_lim=(-1,1),max_iter=50)
roots=find_roots(z)
show_fractal(rr,ii,z,iterations,roots,40)

Sie haben nun alle Werkzeuge um das Newton-Verfahren anzuwenden und die Newton-Fraktale zu plotten. Bearbeiten Sie nun die Aufgaben unten. Dafür können Sie in diesem Notebook mit verschiedenen Funktionen experimentieren, um so auf die Lösungen zu kommen.

**Tipp:** Die imaginäre Einheit wird in Python mit <code>1j</code> beschrieben. [Hier](https://www.audiolabs-erlangen.de/resources/MIR/FMP/C2/C2_ComplexNumbers.html) gibt's sonst noch ein paar Beispiele wie Sie in Python mit komplexen Zahlen arbeiten können.

### Aufgabe: Fraktale

In den bisherigen Beispielen konnten wir stets fraktales Verhalten beobachten, d.h. es treten Fraktale im Plot auf. Dies sind gerade die verschachtelten Regionen, die wir in den Plots beobachten konnten. Welche Vorraussetzung muss eine Funktion erfüllen, damit wir gerade Fraktale beobachten können? Von den folgenden Optionen ist genau eine korrekt. Erstellen Sie Plots verschiedener Funktionen, um herauszufinden welche. 

1. Tritt immer auf
2. Tritt ausschließlich bei Polynomen auf
3. Tritt immer auf bei Polynomen mit mindestens drei (paarweise verschiedenen) Nullstellen
4. Tritt immer auf bei Polynomen von Grad $\geq 3$
5. Tritt immer auf bei Polynomen deren Koeffizienten alle reell sind

**Ihre Lösung hier**

Anleitung: Doppelklick auf die Zelle und dann ein <code>x</code> in das richtige Kästchen <code>[x]</code>. Wenn Sie nicht damit zurecht kommen, löschen Sie einfach alle falschen Antworten.

- [ ] Tritt immer auf
- [ ] Tritt ausschließlich bei Polynomen auf
- [ ] Tritt immer auf bei Polynomen mit mindestens drei (paarweise verschiedenen) Nullstellen
- [ ] Tritt immer auf bei Polynomen von Grad $\geq 3$
- [ ] Tritt immer auf bei Polynomen deren Koeffizienten alle reell sind


In [ ]:
### Platz für Ihren Code zum Experimentieren

**Hintergrund und Ausblick**

_folgt in der Lösung, also schauen Sie sich die unbedingt auch noch an_

### Aufgabe: Achsensymmetrie

Bisher waren alle Newton-Fraktale symmetrisch zur x-Achse. Wenn wir den Plot also an der x-Achse spiegeln, kommt bis auf unterschiedliche Farben wieder der selbe Plot heraus.
Finden Sie eine Funktion, deren Plot nicht symmetrisch zur x-Achse ist? Geben Sie die Funktion an (inkl. passendem Plot) oder begründen Sie, weshalb eine solche Funktion nicht existiert.

In [ ]:
### BEGIN SOLUTION
# Ihre Lösung hier
### END SOLUTION

**Erklärung**

_folgt in der Lösung, also schauen Sie sich die unbedingt auch noch an_

### Aufgabe: Rotationssymmetrie

Der Plot von $z\mapsto z^3-1$ hat eine Rotationssymmetrie um 120°. D.h, wenn wir unseren Plot um 120° drehen, haben wir -- abgesehen von verschiedenen Farben -- den selben Plot. Finden Sie ein Polynom mit 90°-Rotationssymmetrie? Geben Sie die Funktion an (inkl. passendem Plot) oder begründen Sie, weshalb eine solche Funktion nicht existiert.

In [ ]:
### BEGIN SOLUTION
# Ihre Lösung hier
### END SOLUTION

**Erklärung**

_folgt in der Lösung, also schauen Sie sich die unbedingt auch noch an_